파일 불러오기(인코딩이 다름..)

In [4]:
import pandas as pd

# CSV 불러오기
# df = pd.read_csv("/data1/workspace/gold_standardized.csv", encoding="utf-8")  # 필요시 cp949, euc-kr 등으로 변경

# 수정된 코드
df = pd.read_csv('/data1/workspace/prepro_calcul/t4.csv', encoding='cp949')
# 또는
# df = pd.read_csv('파일명.csv', encoding='euc-kr')

In [5]:
print(df.head())

                                               title  \
0  wee1 confers resistance to krasg12c inhibitors...   
1  palbociclib overcomes afatinib resistance in n...   
2  regorafenib suppresses epidermal growth factor...   
3  her2 exon 20 insertions in non-small-cell lung...   
4  xie-bai-san increases nsclc cells sensitivity ...   

                                                drug  \
0          ['sotorasib', 'adavosertib', 'puromycin']   
1           ['palbociclib', 'letrozole', 'afatinib']   
2  ['regorafenib', 'erlotinib', '6bromo(2chloroph...   
3  ['pyrotinib', 'afatinib', 'trastuzumabemtansine']   
4                         ['sirolimus', 'gefitinib']   

                                          normalized  
0            ['sotorasib','adavosertib','puromycin']  
1             ['palbociclib','letrozole','afatinib']  
2  ['regorafenib','erlotinib','6bromo(2chlorophen...  
3    ['pyrotinib','afatinib','trastuzumabemtansine']  
4                          ['sirolimus','gefitinib'

In [6]:
# CSV 불러오기
df1 = pd.read_csv("pred_5_generic.csv", encoding="utf-8")  # 필요시 cp949, euc-kr 등으로 변경

print(df1.head())

                                         single_drug  \
0  palbociclib; afatinib; letrozole; ribociclib; ...   
1                             erlotinib; regorafenib   
2                         pyrotinib; afatinib; T-DM1   
3  gefitinib; rapamycin; chloroquine; 3-methylade...   
4      alectinib; crizotinib; lorlatinib; naquotinib   

                                               title  \
0  Palbociclib overcomes afatinib resistance in n...   
1  Regorafenib suppresses epidermal growth factor...   
2  HER2 exon 20 insertions in non-small-cell lung...   
3  Xie-Bai-San increases NSCLC cells sensitivity ...   
4  Rapid Acquisition of Alectinib Resistance in\n...   

                                   Generic_Drug_List  
0  PALBOCICLIB;AFATINIB;LETROZOLE;RIBOCICLIB;ABEM...  
1                              ERLOTINIB;REGORAFENIB  
2           PYROTINIB;AFATINIB;TRASTUZUMAB EMTANSINE  
3    GEFITINIB;SIROLIMUS;CHLOROQUINE;3-METHYLADENINE  
4         ALECTINIB;CRIZOTINIB;LORLATINIB;NAQUOTINI

비교셋과 정답셋 정규화

In [7]:
import pandas as pd

# 1. 각 파일을 적절한 인코딩으로 읽기
expert_df = pd.read_csv("/data1/workspace/gold_standardized _doublecheck.csv", encoding='cp949')   # 한글 윈도우용 CSV
llm_df = pd.read_csv("pred_5_generic.csv", encoding="utf-8")        # UTF-8 CSV

# 2. 정규화 함수 정의
def normalize_drug_list(drug_str):
    if pd.isna(drug_str):
        return []
    drugs = drug_str.split(';')
    drugs = [d.replace('-', '').replace(' ', '').lower() for d in drugs if d.strip() != '']
    return drugs

# 3. 정규화 적용
expert_df['normalized'] = expert_df['single_drug'].apply(normalize_drug_list)
llm_df['normalized'] = llm_df['single_drug'].apply(normalize_drug_list)

# 4. UTF-8로 통합 저장 (옵션)
expert_df.to_csv("expert_drugs_utf8.csv", encoding='utf-8', index=False)
llm_df.to_csv("llm_drugs_utf8.csv", encoding='utf-8', index=False)


FileNotFoundError: [Errno 2] No such file or directory: '/data1/workspace/gold_standardized _doublecheck.csv'

In [ ]:
print(expert_df.head())

                                               title  \
0  conditionally reprogrammed colorectal cancer c...   
1  combining mek and src inhibitors for treatment...   
2  sensitivity of kras-mutant colorectal cancers ...   
3  targeting ras mutant colorectal cancer with du...   
4  mcl-1 inhibition overcomes intrinsic and acqui...   

                                     single_drug  \
0  cetuximab; palbociclib; sorafenib; trametinib   
1                          dasatinib; trametinib   
2                        palbociclib; trametinib   
3                        palbociclib; trametinib   
4                            regorafenib; s63845   

                                 generic_drug  \
0  cetuximab;palbociclib;sorafenib;trametinib   
1                        dasatinib;trametinib   
2                      palbociclib;trametinib   
3                      palbociclib;trametinib   
4                          regorafenib;s63845   

                                        normalized  
0 

In [ ]:
print(llm_df.head())

                                         single_drug  \
0  palbociclib; afatinib; letrozole; ribociclib; ...   
1                             erlotinib; regorafenib   
2                         pyrotinib; afatinib; T-DM1   
3  gefitinib; rapamycin; chloroquine; 3-methylade...   
4      alectinib; crizotinib; lorlatinib; naquotinib   

                                               title  \
0  Palbociclib overcomes afatinib resistance in n...   
1  Regorafenib suppresses epidermal growth factor...   
2  HER2 exon 20 insertions in non-small-cell lung...   
3  Xie-Bai-San increases NSCLC cells sensitivity ...   
4  Rapid Acquisition of Alectinib Resistance in\n...   

                                   Generic_Drug_List  \
0  PALBOCICLIB;AFATINIB;LETROZOLE;RIBOCICLIB;ABEM...   
1                              ERLOTINIB;REGORAFENIB   
2           PYROTINIB;AFATINIB;TRASTUZUMAB EMTANSINE   
3    GEFITINIB;SIROLIMUS;CHLOROQUINE;3-METHYLADENINE   
4         ALECTINIB;CRIZOTINIB;LORLATINIB;NAQU

리스트 제거

In [ ]:
import ast
import pandas as pd
import numpy as np

def list_or_literal_to_str(x, delimiter=';'):
    # NaN 처리
    if pd.isna(x):
        return ""
    # 이미 리스트인 경우
    if isinstance(x, list):
        cleaned = [str(i).strip() for i in x if str(i).strip() != ""]
        return delimiter.join(cleaned)
    # 문자열인 경우: "a;b" 은 그대로, "['a', 'b']" 같은 리터럴이면 파싱
    if isinstance(x, str):
        s = x.strip()
        # 빈 문자열
        if s == "":
            return ""
        # 이미 세미콜론으로 구분된 정상 문자열이면 그대로 (단 공백 정리)
        if delimiter in s:
            parts = [p.strip() for p in s.split(delimiter) if p.strip() != ""]
            return delimiter.join(parts)
        # 문자열 리터럴 형태인지 시도 파싱 (예: "['a','b']" or "['a', 'b']")
        try:
            lit = ast.literal_eval(s)
            if isinstance(lit, (list, tuple)):
                cleaned = [str(i).strip() for i in lit if str(i).strip() != ""]
                return delimiter.join(cleaned)
        except Exception:
            pass
        # 그 외 일반 문자열: 공백 제거 후 소문자화(필요시)
        return s.strip()
    # 그 외 타입은 문자열로 변환
    return str(x).strip()

# 예: expert_df와 llm_df에 적용
# expert_df['normalized'] = expert_df['normalized'].apply(list_or_literal_to_str)
llm_df['normalized'] = llm_df['normalized'].apply(list_or_literal_to_str)

# 확인용 출력 (상위 5개)
print("=== expert_df normalized sample ===")
print(expert_df['normalized'].head())
print("=== llm_df normalized sample ===")
print(llm_df['normalized'].head())

# 필요하면 바로 semicolon 기준으로 다시 리스트로 비교할 때:
# expert_df['normalized_list'] = expert_df['normalized'].apply(lambda s: [] if s=="" else [i.strip() for i in s.split(';')])
llm_df['normalized_list'] = llm_df['normalized'].apply(lambda s: [] if s=="" else [i.strip() for i in s.split(';')])


NameError: name 'llm_df' is not defined